In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import datetime
import os
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Leitura do dataset para treino da LSTM (RNN) em séries temporais

In [2]:
RANDOM_SEED=21

model = "gru"
src_type = "regular"

dir_results = f"../../data/results/{src_type}"
dir_figures = f"{dir_results}/figures/{model}"

if not os.path.exists(dir_figures):
    os.makedirs(dir_figures)

path_datasets = "../../data/datasets"
dataset = "Itaipu_POC_VAZAO_V1.csv"

## Número de Semanas Operativas Retroativas a serem utilizadas no Treinamento dos Algoritmos
n = 6

In [3]:
df_ts = pd.read_csv(f'{path_datasets}/{dataset}', index_col='time')
df_ts

,bacia_prec_sum,vazao_itaipu
time,,
2001-01-06,188551.1250,96628.00
2001-01-13,431305.6250,113552.00
2001-01-20,199386.4375,84168.00
2001-01-27,337050.2500,81859.00
2001-02-03,369936.3125,98723.00
...,...,...
2020-12-05,314893.7500,37020.23
2020-12-12,381038.8750,46404.99
2020-12-19,362357.2500,63216.66


In [4]:
# precisamos resgatar o índice/data
new_df = df_ts.copy().reset_index(drop=True)
new_df['time'] = df_ts.index
new_df = new_df[['time', 'bacia_prec_sum', 'vazao_itaipu']]
new_df

,time,bacia_prec_sum,vazao_itaipu
0,2001-01-06,188551.1250,96628.00
1,2001-01-13,431305.6250,113552.00
2,2001-01-20,199386.4375,84168.00
3,2001-01-27,337050.2500,81859.00
4,2001-02-03,369936.3125,98723.00
...,...,...,...
1042,2020-12-05,314893.7500,37020.23
1043,2020-12-12,381038.8750,46404.99
1044,2020-12-19,362357.2500,63216.66
1045,2020-12-26,309619.8750,59164.67


In [5]:
new_df = new_df.values
new_df

array([['2001-01-06', 188551.125, 96628.0],
       ['2001-01-13', 431305.625, 113552.0],
       ['2001-01-20', 199386.4375, 84168.0],
       ...,
       ['2020-12-19', 362357.25, 63216.66000000001],
       ['2020-12-26', 309619.875, 59164.67],
       ['2021-01-02', 259895.3125, 71591.0]], dtype=object)

### Divisão dos datasets em séries temporais e treino e teste

In [6]:
#Divide uma sequencia multivariável em amostras
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix >= len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, [0,2]]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [7]:
len(new_df)

1047

In [8]:
### Para debugar a função split_sequences
sequences = new_df
i=0
end_ix=i+n

sequences[i:end_ix, :], sequences[end_ix, [0,2]]

(array([['2001-01-06', 188551.125, 96628.0],
        ['2001-01-13', 431305.625, 113552.0],
        ['2001-01-20', 199386.4375, 84168.0],
        ['2001-01-27', 337050.25, 81859.0],
        ['2001-02-03', 369936.3125, 98723.0],
        ['2001-02-10', 371198.8125, 96852.0]], dtype=object),
 array(['2001-02-17', 130217.0], dtype=object))

In [9]:
X, y = split_sequences(new_df, n)
X.shape, y.shape

((1041, 6, 3), (1041, 2))

In [10]:
seq = 2
X[seq], y[seq]

(array([['2001-01-20', 199386.4375, 84168.0],
        ['2001-01-27', 337050.25, 81859.0],
        ['2001-02-03', 369936.3125, 98723.0],
        ['2001-02-10', 371198.8125, 96852.0],
        ['2001-02-17', 482565.125, 130217.0],
        ['2001-02-24', 385677.75, 116785.0]], dtype=object),
 array(['2001-03-03', 86866.0], dtype=object))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((832, 6, 3), (209, 6, 3), (832, 2), (209, 2))

In [12]:
seq = 0
X_train[:,:,1:][seq], y_train[:,1][seq]

(array([[158343.875, 80201.0],
        [1392.375, 68083.0],
        [23606.375, 57768.0],
        [13071.5, 53322.0],
        [9709.5, 50117.0],
        [34233.75, 49400.0]], dtype=object),
 48115.0)

### Normalização dos dados de treino/teste

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Reshape the data to 2D
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
y_train_reshaped = y_train.reshape(-1, y_train.shape[-1])
y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])

# Initialize MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit and transform on the training data (ignoring dates) # _ stands for normalized data
X_train_ = scaler_X.fit_transform(X_train_reshaped[:,1:])
y_train_ = scaler_y.fit_transform(y_train_reshaped[:,1:])

# Transform the test data (ignoring dates)
X_test_ = scaler_X.transform(X_test_reshaped[:,1:])
y_test_ = scaler_y.transform(y_test_reshaped[:,1:])

# Reshape back to the original shape (ignoring dates)
X_train_ = X_train_.reshape(X_train[:,:,1:].shape).astype('float32')
X_test_ = X_test_.reshape(X_test[:,:,1:].shape).astype('float32')
y_train_ = y_train_.reshape(y_train[:,1:].shape).astype('float32')
y_test_ = y_test_.reshape(y_test[:,1:].shape).astype('float32')

### GRU

In [14]:
now = datetime.datetime.now().strftime('%Y%m%d') # _%Hh%M
modelo_numerico = 'so_prev' # previsão para a semana operacional seguinte

dir_rna = f'{dir_results}/rna/{modelo_numerico}_{now}'
if not os.path.exists(dir_rna):
    os.makedirs(dir_rna)

file_ann = f'{dir_rna}/gru_{modelo_numerico}.h5' 
best_file_ann = f'{dir_rna}/best_gru_{modelo_numerico}.h5' 

In [15]:
monitor_metric = 'val_mean_absolute_error'
patience=15
n_neurons = 256
max_epochs = 500
n_hidden_layers = 1

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.GRU(
        units=80, 
        activation='relu', 
        return_sequences=True,
        input_shape=[*X_train_.shape[1:]]
    ),
    tf.keras.layers.GRU(
        units=60, 
        activation='relu'
    ),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.losses.MeanAbsoluteError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=patience, 
        mode='min',
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=best_file_ann, 
        monitor=monitor_metric,
        verbose=True, 
        save_best_only=True
        )  
    ]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 6, 80)             20160     
                                                                 
 gru_1 (GRU)                 (None, 60)                25560     
                                                                 
 dense (Dense)               (None, 1)                 61        
                                                                 
Total params: 45,781
Trainable params: 45,781
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(
    X_train_,
    y_train_,
    epochs=max_epochs,
    verbose=True,
    validation_split=0.2,
    callbacks=callbacks,
) 

Epoch 1/500


18/21 [========================>.....] - ETA: 0s - loss: 0.1468 - mean_absolute_error: 0.1468 
Epoch 1: val_mean_absolute_error improved from inf to 0.09024, saving model to ../../data/results/regular/rna/so_prev_20240201/best_gru_so_prev.h5
21/21 [==============================] - 4s 30ms/step - loss: 0.1406 - mean_absolute_error: 0.1406 - val_loss: 0.0902 - val_mean_absolute_error: 0.0902
Epoch 2/500
18/21 [========================>.....] - ETA: 0s - loss: 0.0801 - mean_absolute_error: 0.0801
Epoch 2: val_mean_absolute_error improved from 0.09024 to 0.07046, saving model to ../../data/results/regular/rna/so_prev_20240201/best_gru_so_prev.h5
21/21 [==============================] - 0s 9ms/step - loss: 0.0786 - mean_absolute_error: 0.0786 - val_loss: 0.0705 - val_mean_absolute_error: 0.0705
Epoch 3/500
12/21 [================>.............] - ETA: 0s - loss: 0.0678 - mean_absolute_error: 0.0678
Epoch 3: val_mean_absolute_error did not improve from 0.07046
21/21 [=======================

In [19]:
model.save(file_ann) # salva o modelo atual

### importamos o modelo que melhor performou em 'monitor_metric' durante o treinamento para analisar

In [20]:
model = tf.keras.models.load_model(best_file_ann) # importamos o modelo que melhor performou em 'monitor_metric' durante o treinamento para analisar
# model = tf.keras.models.load_model(file_ann)

In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'],
                         mode='lines',
                         name='Train Loss'))

fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'],
                         mode='lines',
                         name='Validation Loss'))

fig.update_layout(title='Training and Validation Loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss',
                  legend=dict(x=0, y=1, traceorder='normal'),
                  width=900, height=600)

fig.write_image(f"{dir_figures}/rna_training_validation_loss_plot.png")

fig.show()

### Retoma a transformação

In [22]:
y_pred_ = model.predict(X_test_).astype('float32')
y_pred = scaler_y.inverse_transform(y_pred_)

7/7 [==============================] - 0s 2ms/step


In [23]:
# y_test = y_test.reshape(-1, y_test.shape[-1])

mae = mean_absolute_error(y_test[:,1].astype('float32'), y_pred)
mse = mean_squared_error(y_test[:,1].astype('float32'), y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test[:,1].astype('float32'), y_pred)
corr = np.corrcoef(y_test[:,1].astype('float32').T, y_pred.T)[0, 1]

metrics_df = pd.DataFrame(
    columns=['MAE', 'MSE', 'RMSE', 'R2', 'Corr'],
    index=['Decision Tree']
)

metrics_df['MAE'] = mae
metrics_df['MSE'] = mse
metrics_df['RMSE'] = rmse
metrics_df['R2'] = r2
metrics_df['Corr'] = corr
metrics_df

,MAE,MSE,RMSE,R2,Corr
Decision Tree,8287.448242,144067808.0,12002.825195,0.862569,0.929753


In [24]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=y_test[:,1:].ravel(),
        y=y_pred.ravel(),
        mode='markers',
        marker=dict(color='blue', opacity=0.5, line=dict(color='black', width=1)),
        name='Measured vs Predicted'
    )
)


fig.add_trace(
    go.Scatter(
        x=[y_pred.min(), y_pred.max()],
        y=[y_pred.min(), y_pred.max()],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Identity Line'
    )
)

fig.update_layout(
    title='Measured vs Predicted',
    xaxis=dict(title='y_true'),
    yaxis=dict(title='y_pred'),
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=0, r=0, b=0, t=40),
    showlegend=True
)

fig.write_image(f"{dir_figures}/scattered_measured_vs_predicted_plot.png")

fig.show()


In [25]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_ts.index,
        y=df_ts.vazao_itaipu.values, # vazão observada
        mode='lines',
        name='Vazão observada',
    )
)

fig.add_trace(
    go.Scatter(
        x=y_test[:,0],
        y=y_pred.ravel(), # vazão prevista
        mode='markers',
        name='Forecast',
    )
)

fig.update_layout(title=f'Predição - Itaipu')

fig.write_image(f"{dir_figures}/history_measured_vs_predicted_plot.png", width=1400, scale=1)

fig.show()

### Cálculo da importância de variáveis

In [26]:
# def gradient_importance(seq, model):
#     seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)
#     with tf.GradientTape() as tape:
#         predictions = model(seq)
#     grads = tape.gradient(predictions, seq)
#     grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
#     return grads

In [27]:
# X_train[0,:,1:]

In [28]:
# gradient_importance(X_train[0,:,1:], model)

In [29]:
# importances = []
# for i in range(0, X_train.shape[0]):
#     importances.append(gradient_importance(X_train[i,:,1:], model))

In [30]:
# importance = np.mean(np.array(importances), axis=0)
# importance

In [31]:
# plt.figure(figsize=(6,4))
# plt.title('Importância das Variáveis')

# plt.bar(df_ts.columns.values[:importance.shape[0]],importance.tolist())

# plt.savefig(f"{dir_figures}/feature_importance.png")
# plt.show()